In [1]:
import gspread
from google.oauth2.service_account import Credentials
import re
import configparser

In [4]:
# return list of source sheets
def source_sheets(contact_config):
    sheet_info = []
    for key in contact_config.keys():
        if 'source_sheet' in key:
            sheet, column, row = contact_config.get(key).split(',')
            sheet_info.append( (sheet, int(column), int(row)) )
    return tuple(sheet_info)

# split emails if they put two in a cell
def split_emails(raw_list):
    split_result = []
    for entry in raw_list:
        # Split on anything that isn't a valid email character
        parts = re.split(r'[^a-zA-Z0-9._%+\-@]+', entry)
        split_result.extend([p.strip() for p in parts if p.strip()])
    return split_result

# Basic email pattern: username@domain
EMAIL_REGEX = re.compile(r"^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$")
def is_valid_email(email):
    return EMAIL_REGEX.match(email) is not None

In [5]:
# get config
cp = configparser.ConfigParser() 
cp.read('config.ini')
config = dict(cp['contacts'])

# Setup auth and client
SCOPES = ['https://www.googleapis.com/auth/drive']
creds = Credentials.from_service_account_file(config.get('credentials'), scopes=SCOPES)
client = gspread.authorize(creds)

In [23]:
# Open your spreadsheet by name or URL
spreadsheet = client.open(config.get('workbook'))
sheets = source_sheets( config )
sheets

(('AGMB Roster', 6, 1),
 ('AGIMBA Org', 2, 1),
 ('AG Staff', 3, 1),
 ('AGIMBA Contacts', 3, 1),
 ('MS CG Roster', 6, 1),
 ('HS CG Roster', 6, 1),
 ('Percussion', 9, 1))

## Build lists for color guard

In [42]:
sheet_email_lists = spreadsheet.worksheet(config.get('email_sheet'))

# get MS CG Roster emails and build list of lists as need for sheet update
(sheet, column, row) = [t for t in sheets if t[0] == 'MS CG Roster'][0]
ms_cg_emails = split_emails( spreadsheet.worksheet(sheet).col_values(column)[row:] )
ms_cg_emails_cleaned = {email.lower() for email in ms_cg_emails if is_valid_email(email) }
ms_cg_update_list = [ [email,] for email in sorted(ms_cg_emails_cleaned) ]

# get HS CG Roster emails and build list of lists as need for sheet update
(sheet, column, row) = [t for t in sheets if t[0] == 'HS CG Roster'][0]
hs_cg_emails = split_emails( spreadsheet.worksheet(sheet).col_values(column)[row:] )
hs_cg_emails_cleaned = {email.lower() for email in hs_cg_emails if is_valid_email(email) }
hs_cg_update_list = [ [email,] for email in sorted(hs_cg_emails_cleaned) ]

# build the combined list
cg_combined = set()
cg_combined.update( {email for email in hs_cg_emails_cleaned } )
cg_combined.update( {email for email in ms_cg_emails_cleaned } )
cg_combined_update_list = [ [email,] for email in sorted(cg_combined) ]

# Clear old data
sheet_email_lists.batch_clear(['C2:C',])
sheet_email_lists.batch_clear(['D2:D',])
sheet_email_lists.batch_clear(['E2:E',])

# Write individual lists to their respective columns
sheet_email_lists.update(range_name=f'C2:C{len(indoor_combined_update_list)+1}', values=indoor_combined_update_list )
sheet_email_lists.update(range_name=f'D2:D{len(ms_cg_update_list)+1}', values=ms_cg_update_list )
sheet_email_lists.update(range_name=f'E2:E{len(hs_cg_update_list)+1}', values=hs_cg_update_list )


{'spreadsheetId': '1SffcFMoEWm68vFWWJcJXMEFZ1OO0v56F0PXqSuBArtg',
 'updatedRange': 'EmailLists!E2:E14',
 'updatedRows': 13,
 'updatedColumns': 1,
 'updatedCells': 13}

## Build the lists for percussion

In [43]:
sheet_email_lists = spreadsheet.worksheet(config.get('email_sheet'))

# get Percussion Roster emails and build list of lists as need for sheet update
(sheet, column, row) = [t for t in sheets if t[0] == 'Percussion'][0]
percussion_emails = split_emails( spreadsheet.worksheet(sheet).col_values(column)[row:] )
percussion_emails_cleaned = {email.lower() for email in percussion_emails if is_valid_email(email) }
percussion_update_list = [ [email,] for email in sorted(percussion_emails_cleaned) ]

# Clear old data
sheet_email_lists.batch_clear(['F2:F',])

# Write individual lists to their respective columns
sheet_email_lists.update(range_name=f'F2:F{len(percussion_update_list)+1}', values=percussion_update_list )


{'spreadsheetId': '1SffcFMoEWm68vFWWJcJXMEFZ1OO0v56F0PXqSuBArtg',
 'updatedRange': 'EmailLists!F2:F30',
 'updatedRows': 29,
 'updatedColumns': 1,
 'updatedCells': 29}

## Build full indoor list

In [44]:
# build the combined list
indoor_combined = set()
indoor_combined.update( {email for email in hs_cg_emails_cleaned } )
indoor_combined.update( {email for email in ms_cg_emails_cleaned } )
indoor_combined.update( {email for email in percussion_emails_cleaned } )
indoor_combined_update_list = [ [email,] for email in sorted(indoor_combined) ]

# Clear old data
sheet_email_lists.batch_clear(['B2:B',])

# Write individual lists to their respective columns
sheet_email_lists.update(range_name=f'B2:B{len(indoor_combined_update_list)+1}', values=indoor_combined_update_list )

{'spreadsheetId': '1SffcFMoEWm68vFWWJcJXMEFZ1OO0v56F0PXqSuBArtg',
 'updatedRange': 'EmailLists!B2:B63',
 'updatedRows': 62,
 'updatedColumns': 1,
 'updatedCells': 62}

## Build full AGIMBA List

In [20]:
# Grab all the emails from all the sheets
emails_combined = set()
for sheet, column, row in sheets:
    emails = split_emails( spreadsheet.worksheet(sheet).col_values(column)[row:] )
    emails_combined.update( {email.lower() for email in emails if is_valid_email(email) } )
    
#  Build list of lists as need for sheet update
update_list = [ [email,] for email in sorted(emails_combined) ]

# Clear old data
sheet_email_lists = spreadsheet.worksheet(config.get('email_sheet'))
sheet_email_lists.batch_clear([config.get('email_range')])

# Prepare for writing
sheet_email_lists.update(range_name=f'{config.get('email_range')}{len(update_list)+1}', values=update_list )

print(f"Wrote {len(update_list)} unique emails to '{sheet_email_lists.title}' sheet.")

Wrote 138 unique emails to 'EmailLists' sheet.


In [8]:
sheet_email_lists.title

'EmailLists'

In [9]:
config.get('email_range')

'A2:A'